In [ ]:
# ===============================================================
# BUSCA EM GRADE COM PARÂMETROS OTIMIZADOS
# ===============================================================
# COMENTÁRIO INICIAL: Este código implementa extensos blocos try-except
# para garantir robustez numérica e previnir falhas em problemas complexos.
# As estratégias de tratamento de erro incluem:
# - Verificação de valores numéricos válidos
# - Limites para evitar overflow/underflow
# - Tratamento de divisões por zero
# - Recuperação de estados anteriores em caso de falha
# - Continuação da busca mesmo com erros em combinações específicas
# ===============================================================
import pycutest as pc
import numpy as np
import time
import pandas as pd

# ===============================================================
# Método do Gradiente Descendente com Armijo
# ===============================================================
def steepest_descent_armijo(p, x0,
                            tau=1e-6,
                            maxit=1000,
                            c1=1e-4,
                            rho=0.5,
                            alpha_init=1.0):
    """
    Implementação robusta do método de gradiente descendente com Armijo.
    """
    try:
        # Inicialização das variáveis
        xk = x0.copy()
        fk, gk = p.obj(xk, gradient=True)
        norm_g = np.linalg.norm(gk)
        hist = [(0, fk, norm_g)]  # Histórico de iterações

        start_time = time.time()
        f_evals = 1  # Contador de avaliações da função
        g_evals = 1  # Contador de avaliações do gradiente
        line_searches = 0  # Contador de buscas de linha

        # Loop principal de otimização
        for k in range(1, maxit + 1):
            # Verificar critério de parada
            if norm_g <= tau * (1 + abs(fk)):
                break

            alpha = alpha_init
            gTdk = -norm_g**2  # Produto interno gradiente × direção

            inner_iter = 0
            max_inner = 500  # Número máximo de iterações internas
            step_found = False  # Flag para passo encontrado

            # Busca de linha com limites estritos
            while inner_iter < max_inner and not step_found:
                try:
                    x_new = xk - alpha * gk  # Novo ponto candidato
                    f_new = p.obj(x_new)  # Avalia função no novo ponto
                    f_evals += 1

                    # Verifica condição de Armijo
                    if f_new <= fk + c1 * alpha * gTdk:
                        step_found = True  # Passo aceito
                    else:
                        alpha *= rho  # Reduz tamanho do passo
                        line_searches += 1
                        inner_iter += 1

                    # Condição de passo mínimo
                    if alpha < 1e-15:
                        break

                except Exception as e:
                    print(f"      Erro em busca de linha: {e}")
                    break

            # Se não encontrou passo válido, usa o melhor disponível
            if not step_found:
                x_new = xk - alpha * gk
                try:
                    f_new = p.obj(x_new)
                    f_evals += 1
                except:
                    # Se falhar, mantém o ponto atual
                    x_new = xk.copy()
                    f_new = fk

            # Verificar se há progresso
            if np.linalg.norm(x_new - xk) < 1e-15 and not step_found:
                break

            # Atualizar ponto
            xk_prev = xk.copy()  # Backup do ponto anterior
            xk = x_new

            try:
                # Avaliar novo ponto
                fk, gk = p.obj(xk, gradient=True)
                f_evals += 1
                g_evals += 1
                norm_g = np.linalg.norm(gk)
            except Exception as e:
                print(f"      Erro avaliando ponto novo: {e}")
                # Reverter ao ponto anterior
                xk = xk_prev
                break

            hist.append((k, fk, norm_g))  # Atualizar histórico

            # Verificar estagnação
            if k > 5 and len(hist) > 6:
                recent_improvement = abs(hist[-1][1] - hist[-6][1])
                if recent_improvement < 1e-12:
                    break

        elapsed_time = time.time() - start_time
        return xk, fk, hist, elapsed_time, f_evals, g_evals, line_searches

    except Exception as e:
        print(f"      Erro crítico em Armijo: {e}")
        # Retornar valores padrão em caso de erro
        return x0, p.obj(x0), [(0, p.obj(x0), np.linalg.norm(p.grad(x0)))], 0.0, 1, 1, 0

# ===============================================================
# Método GBB (PARÂMETROS OTIMIZADOS)
# ===============================================================
def gbb_raydan(p, x0, tau=1e-6, epsilon=1e-10, maxit=1000, M=10,
               gamma=0.2, y_gbb=-1e-4, alpha_init=1.0):
    """
    Implementação robusta do método GBB.
    """
    try:
        xk = x0.copy()
        fk, gk = p.obj(xk, gradient=True)
        alpha_k = alpha_init  # Passo espectral inicial
        fk_list = [fk]  # Lista para critério não monótono
        norm_g = np.linalg.norm(gk)
        hist = [(0, fk, norm_g)]

        start_time = time.time()
        f_evals = 1
        g_evals = 1
        line_searches = 0
        spectral_searches = 0  # Contador de buscas espectrais

        # Loop principal do algoritmo GBB
        for k in range(1, maxit + 1):
            # Critério de parada
            if norm_g <= tau * (1 + abs(fk)) or norm_g < 1e-14:
                break

            # Limitação robusta de alpha_k
            if alpha_k <= epsilon or alpha_k >= 1/epsilon or np.isnan(alpha_k):
                alpha_k = 1.0  # Reset para valor padrão

            lamda = 1 / alpha_k  # Parâmetro do passo

            # Critério de decréscimo não monótono
            min_k_M = min(k, M)
            f_ref = np.max(fk_list[-min_k_M:])  # Valor de referência

            inner_iter = 0
            max_inner = 500  # Máximo de iterações internas
            step_accepted = False  # Flag de passo aceito

            # Loop interno de busca de linha
            while inner_iter < max_inner and not step_accepted:
                try:
                    x_new = xk - lamda * gk
                    f_new = p.obj(x_new)
                    f_evals += 1

                    # Condição de aceitação não monótona
                    if f_new <= f_ref + y_gbb * lamda * norm_g**2:
                        # Passo aceito
                        lamda_k = lamda
                        g_new = p.grad(x_new)
                        g_evals += 1

                        y_km1 = g_new - gk  # Diferença de gradientes

                        # Cálculo seguro de alpha_k
                        denom = lamda_k * np.dot(gk, gk)
                        if abs(denom) > 1e-12 and not np.isnan(denom):
                            alpha_k = -np.dot(gk, y_km1) / denom
                            spectral_searches += 1
                        else:
                            alpha_k = 1.0  # Valor padrão em caso de problema

                        # Atualização das variáveis
                        xk = x_new
                        fk = f_new
                        gk = g_new
                        norm_g = np.linalg.norm(gk)
                        fk_list.append(fk)
                        step_accepted = True

                    else:
                        # Redução de passo
                        lamda *= gamma
                        alpha_k = 1.0 / lamda
                        line_searches += 1

                    inner_iter += 1

                    # Condição de passo mínimo
                    if lamda < 1e-15:
                        break

                except Exception as e:
                    print(f"      Erro em busca GBB: {e}")
                    break

            if not step_accepted:
                break

            hist.append((k, fk, norm_g))

            # Verificar estagnação
            if k > 5 and len(hist) > 6:
                recent_improvement = abs(hist[-1][1] - hist[-6][1])
                if recent_improvement < 1e-12:
                    break

        elapsed_time = time.time() - start_time
        return xk, fk, hist, elapsed_time, f_evals, g_evals, line_searches, spectral_searches

    except Exception as e:
        print(f"      Erro crítico em GBB: {e}")
        # Retornar valores padrão em caso de erro
        f0, g0 = p.obj(x0, gradient=True)
        return x0, f0, [(0, f0, np.linalg.norm(g0))], 0.0, 1, 1, 0, 0

# ===============================================================
# BUSCA EM GRADE COM PARÂMETROS OTIMIZADOS
# ===============================================================
def grid_search_optimization(problema_nombre):
    """
    Executa busca em grade com parâmetros otimizados para cada algoritmo.
    """
    print("\n" + "#"*60)
    print(f"## BUSCA EM GRADE COM PARÂMETROS OTIMIZADOS: {problema_nombre}")
    print("#"*60)

    try:
        # Carregar problema de otimização
        p = pc.import_problem(problema_nombre)
        print(f"Problema carregado: {p.name}, Dimensão: {p.n}")
    except Exception as e:
        print(f"Erro ao carregar o problema: {e}")
        return None, None, None

    x0 = p.x0.copy()
    try:
        # Avaliação inicial
        f0, g0 = p.obj(x0, gradient=True)
        print(f"Valor inicial f(x0): {f0:.6e}")
        print(f"Norma inicial do gradiente: {np.linalg.norm(g0):.3e}")
    except Exception as e:
        print(f"Erro avaliando função inicial: {e}")
        return None, None, None

    # ===============================================================
    # PARÂMETROS OTIMIZADOS BASEADOS NA LITERATURA
    # ===============================================================

    # PARÂMETROS ÓTIMOS PARA ARMIJO (baseados em pesquisa)
    armijo_optimized_grid = {
        'c1': [1e-6, 1e-4],        # Valores ótimos para condição Armijo
        'rho': [0.5, 0.7],         # Fatores de redução balanceados
        'tau': [1e-8, 1e-6],       # Tolerâncias progressivas
        'alpha_init': [0.5, 1.0]   # Passos iniciais variados
    }

    # PARÂMETROS ÓTIMOS PARA GBB (baseados em Raydan 1997)
    gbb_optimized_grid = {
        'M': [5, 10, 15],          # Janelas não monótonas ótimas
        'tau': [1e-8, 1e-6],       # Tolerâncias progressivas
        'epsilon': [1e-10, 1e-12], # Limites para alpha_k
        'gamma': [0.2, 0.25],      # Fatores de redução ótimos
        'y_gbb': [-1e-5, -1e-4],   # Parâmetros de aceitação
        'alpha_init': [0.5, 1.0]   # Passos iniciais espectrais
    }

    all_results = []  # Todos os resultados
    armijo_results = []  # Resultados do Armijo
    gbb_results = []  # Resultados do GBB

    # ===============================================================
    # BUSCA EM GRADE - MÉTODO ARMIJO
    # ===============================================================
    print("\n" + "="*50)
    print("BUSCA COM PARÂMETROS OTIMIZADOS - ARMIJO")
    print("="*50)

    armijo_count = 0
    # Calcula o número total de combinações possíveis para Armijo
    total_armijo_combinations = (len(armijo_optimized_grid['c1']) *
                                len(armijo_optimized_grid['rho']) *
                                len(armijo_optimized_grid['tau']) *
                                len(armijo_optimized_grid['alpha_init']))

    print(f"Total combinações Armijo: {total_armijo_combinations}")

    # Loop sobre todas as combinações de parâmetros do Armijo
    # Estratégia de busca exaustiva para encontrar a melhor configuração
    for c1 in armijo_optimized_grid['c1']:
        for rho in armijo_optimized_grid['rho']:
            for tau in armijo_optimized_grid['tau']:
                for alpha_init in armijo_optimized_grid['alpha_init']:
                    armijo_count += 1
                    print(f"Armijo [{armijo_count:2d}/{total_armijo_combinations:2d}]: "
                          f"c1={c1:.1e}, rho={rho}, tau={tau:.1e}, α_init={alpha_init}")

                    try:
                        # Executar método Armijo com a combinação atual de parâmetros
                        # Cada combinação representa uma configuração única do algoritmo
                        result = steepest_descent_armijo(
                            p, x0.copy(), tau=tau, c1=c1, rho=rho,
                            alpha_init=alpha_init, maxit=2000
                        )

                        # Verificação de segurança para resultados nulos
                        if result is None:
                            print("   Erro: A função retornou None")
                            continue

                        # Extrair resultados da execução
                        x_opt, f_opt, hist, elapsed_time, f_evals, g_evals, line_searches = result

                        # Verificar se o histórico é válido
                        if not hist or len(hist) == 0:
                            print("   Histórico vazio")
                            continue

                        # Calcular métricas de desempenho
                        final_iter = len(hist) - 1
                        final_norm_g = hist[-1][2]
                        # Critério de convergência: norma do gradiente suficientemente pequena
                        converged = final_norm_g <= tau * (1 + abs(f_opt))

                        # Armazenar resultados detalhados para análise posterior
                        result_dict = {
                            'method': 'Armijo',
                            'params': {
                                'c1': c1,           # Parâmetro da condição Armijo
                                'rho': rho,         # Fator de redução do passo
                                'tau': tau,         # Tolerância de convergência
                                'alpha_init': alpha_init  # Passo inicial
                            },
                            'iterations': final_iter,      # Número total de iterações
                            'final_norm_g': final_norm_g,  # Norma final do gradiente
                            'final_f': f_opt,             # Valor final da função
                            'time': elapsed_time,         # Tempo de execução
                            'f_evals': f_evals,           # Avaliações da função
                            'g_evals': g_evals,           # Avaliações do gradiente
                            'line_searches': line_searches, # Buscas de linha realizadas
                            'spectral_searches': 0,       # Armijo não usa buscas espectrais
                            'converged': converged,       # Flag de convergência
                            'history': hist               # Histórico completo
                        }

                        # Adicionar aos resultados totais e específicos do Armijo
                        all_results.append(result_dict)
                        armijo_results.append(result_dict)

                        # Feedback em tempo real do progresso
                        status = "Convergido" if converged else "Falhou"
                        print(f"   {status} Iter: {final_iter:4d}, ‖∇f‖: {final_norm_g:.2e}, "
                              f"Time: {elapsed_time:.3f}s, LS: {line_searches}")

                    except Exception as e:
                        print(f"   Erro em execução: {e}")
                        continue

    # ===============================================================
    # BUSCA EM GRADE - MÉTODO GBB
    # ===============================================================
    print("\n" + "="*50)
    print("BUSCA COM PARÂMETROS OTIMIZADOS - GBB")
    print("="*50)

    gbb_count = 0
    # Calcula o número total de combinações possíveis para GBB
    # GBB tem mais parâmetros que Armijo, portanto mais combinações
    total_gbb_combinations = (len(gbb_optimized_grid['M']) *
                             len(gbb_optimized_grid['tau']) *
                             len(gbb_optimized_grid['epsilon']) *
                             len(gbb_optimized_grid['gamma']) *
                             len(gbb_optimized_grid['y_gbb']) *
                             len(gbb_optimized_grid['alpha_init']))

    print(f"Total combinações GBB: {total_gbb_combinations}")

    # Loop aninhado sobre todos os parâmetros do GBB
    # Estratégia sistemática para explorar o espaço de parâmetros
    for M in gbb_optimized_grid['M']:
        for tau in gbb_optimized_grid['tau']:
            for epsilon in gbb_optimized_grid['epsilon']:
                for gamma in gbb_optimized_grid['gamma']:
                    for y_gbb in gbb_optimized_grid['y_gbb']:
                        for alpha_init in gbb_optimized_grid['alpha_init']:
                            gbb_count += 1
                            print(f"GBB [{gbb_count:2d}/{total_gbb_combinations:2d}]: "
                                  f"M={M}, τ={tau:.1e}, ε={epsilon:.1e}, γ={gamma}, y={y_gbb:.1e}")

                            try:
                                # Executar método GBB com a combinação atual de parâmetros
                                # Cada execução testa uma configuração única do algoritmo GBB
                                result = gbb_raydan(
                                    p, x0.copy(), tau=tau, epsilon=epsilon, M=M,
                                    gamma=gamma, y_gbb=y_gbb, alpha_init=alpha_init, maxit=2000
                                )

                                # Verificação de segurança
                                if result is None:
                                    print("   Erro: A função retornou None")
                                    continue

                                # Extrair resultados - GBB retorna métricas adicionais
                                x_opt, f_opt, hist, elapsed_time, f_evals, g_evals, line_searches, spectral_searches = result

                                # Validar histórico de execução
                                if not hist or len(hist) == 0:
                                    print("   Histórico vazio")
                                    continue

                                # Calcular métricas de desempenho
                                final_iter = len(hist) - 1
                                final_norm_g = hist[-1][2]
                                # Critério de convergência específico para GBB
                                converged = final_norm_g <= tau * (1 + abs(f_opt))

                                # Estrutura de dados para armazenar resultados do GBB
                                result_dict = {
                                    'method': 'GBB',
                                    'params': {
                                        'M': M,               # Tamanho da janela não monótona
                                        'tau': tau,           # Tolerância de convergência
                                        'epsilon': epsilon,   # Limite inferior para alpha_k
                                        'gamma': gamma,       # Fator de redução do passo
                                        'y_gbb': y_gbb,       # Parâmetro de aceitação
                                        'alpha_init': alpha_init  # Passo inicial espectral
                                    },
                                    'iterations': final_iter,      # Iterações totais
                                    'final_norm_g': final_norm_g,  # Precisão final
                                    'final_f': f_opt,             # Valor ótimo encontrado
                                    'time': elapsed_time,         # Desempenho computacional
                                    'f_evals': f_evals,           # Custo em avaliações de função
                                    'g_evals': g_evals,           # Custo em avaliações de gradiente
                                    'line_searches': line_searches, # Buscas de linha
                                    'spectral_searches': spectral_searches, # Buscas espectrais
                                    'converged': converged,       # Status de convergência
                                    'history': hist               # Dados completos da execução
                                }

                                # Armazenar para análise comparativa
                                all_results.append(result_dict)
                                gbb_results.append(result_dict)

                                # Feedback detalhado do progresso da busca
                                status = "Convergido" if converged else "Falhou"
                                print(f"   {status} Iter: {final_iter:4d}, ‖∇f‖: {final_norm_g:.2e}, "
                                      f"Time: {elapsed_time:.3f}s, LS: {line_searches}, SS: {spectral_searches}")

                            except Exception as e:
                                print(f"   Erro em execução: {e}")
                                continue

    # ===============================================================
    # ANÁLISE COMPARATIVA DETALHADA
    # ===============================================================
    print("\n" + "#"*70)
    print("="*70)
    print("ANÁLISE COMPARATIVA DETALHADA")
    print("="*70)
    print("#"*70)

    def analizar_resultados_optimizados(resultados, nome_algoritmo):
        """Função para análise detalhada dos resultados de cada algoritmo"""
        if not resultados:
            print(f"\nNao ha resultados para {nome_algoritmo}")
            return None

        # Separar resultados convergentes e não convergentes
        convergidos = [r for r in resultados if r['converged']]
        nao_convergidos = [r for r in resultados if not r['converged']]

        print(f"\n{nome_algoritmo.upper()} - Estatisticas:")
        print(f"  Total avaliacoes: {len(resultados)}")
        print(f"  Convergiram: {len(convergidos)} ({len(convergidos)/len(resultados)*100:.1f}%)")
        print(f"  Nao convergiram: {len(nao_convergidos)}")

        if convergidos:
            # Encontrar as melhores configurações por diferentes critérios
            melhor_iter = min(convergidos, key=lambda x: x['iterations'])
            melhor_grad = min(convergidos, key=lambda x: x['final_norm_g'])
            melhor_tempo = min(convergidos, key=lambda x: x['time'])

            print(f"\n  MELHOR POR ITERACOES:")
            print(f"    Parametros: {melhor_iter['params']}")
            print(f"    Iteracoes: {melhor_iter['iterations']}")
            print(f"    ‖∇f‖: {melhor_iter['final_norm_g']:.2e}")
            print(f"    Tempo: {melhor_iter['time']:.3f}s")

            print(f"\n  MELHOR POR GRADIENTE:")
            print(f"    Parametros: {melhor_grad['params']}")
            print(f"    ‖∇f‖: {melhor_grad['final_norm_g']:.2e}")
            print(f"    Iteracoes: {melhor_grad['iterations']}")

            print(f"\n  MELHOR POR TEMPO:")
            print(f"    Parametros: {melhor_tempo['params']}")
            print(f"    Tempo: {melhor_tempo['time']:.3f}s")
            print(f"    Iteracoes: {melhor_tempo['iterations']}")

            return melhor_iter
        else:
            print("  Nenhuma combinacao convergiu")
            if nao_convergidos:
                # Mostrar o melhor entre os não convergentes
                melhor_no_conv = min(nao_convergidos, key=lambda x: x['final_norm_g'])
                print(f"  Melhor entre nao convergentes:")
                print(f"    Parametros: {melhor_no_conv['params']}")
                print(f"    ‖∇f‖: {melhor_no_conv['final_norm_g']:.2e}")
                return melhor_no_conv
            return None

    # Análise para cada algoritmo
    melhor_armijo = analizar_resultados_optimizados(armijo_results, "Armijo")
    melhor_gbb = analizar_resultados_optimizados(gbb_results, "GBB")

    # ===============================================================
    # RECOMENDAÇÕES DE PARÂMETROS ÓTIMOS
    # ===============================================================
    print("\n" + "="*70)
    print("RECOMENDAÇÕES DE PARÂMETROS ÓTIMOS")
    print("="*70)

    if melhor_armijo:
        print(f"\nPARÂMETROS ÓTIMOS RECOMENDADOS - ARMIJO:")
        print(f"  c1 (condicao Armijo): {melhor_armijo['params']['c1']:.1e}")
        print(f"  rho (fator reducao): {melhor_armijo['params']['rho']}")
        print(f"  tau (tolerancia): {melhor_armijo['params']['tau']:.1e}")
        print(f"  alpha_init (passo inicial): {melhor_armijo['params']['alpha_init']}")
        print(f"  Resultados: {melhor_armijo['iterations']} iteracoes, ‖∇f‖ = {melhor_armijo['final_norm_g']:.2e}")

    if melhor_gbb:
        print(f"\nPARÂMETROS ÓTIMOS RECOMENDADOS - GBB:")
        print(f"  M (janela nao monotona): {melhor_gbb['params']['M']}")
        print(f"  tau (tolerancia): {melhor_gbb['params']['tau']:.1e}")
        print(f"  gamma (fator reducao): {melhor_gbb['params']['gamma']}")
        print(f"  y_gbb (parametro aceitacao): {melhor_gbb['params']['y_gbb']:.1e}")
        print(f"  alpha_init (passo inicial): {melhor_gbb['params']['alpha_init']}")
        print(f"  Resultados: {melhor_gbb['iterations']} iteracoes, ‖∇f‖ = {melhor_gbb['final_norm_g']:.2e}")

    # ===============================================================
    # COMPARAÇÃO FINAL ENTRE ALGORITMOS
    # ===============================================================
    print("\n" + "="*70)
    print("COMPARAÇÃO FINAL ENTRE ALGORITMOS")
    print("="*70)

    if melhor_armijo and melhor_gbb:
        print(f"\nCOMPARAÇÃO DIRETA:")
        print(f"Armijo  - Iter: {melhor_armijo['iterations']:4d}, ‖∇f‖: {melhor_armijo['final_norm_g']:.2e}, "
              f"Time: {melhor_armijo['time']:.3f}s")
        print(f"GBB     - Iter: {melhor_gbb['iterations']:4d}, ‖∇f‖: {melhor_gbb['final_norm_g']:.2e}, "
              f"Time: {melhor_gbb['time']:.3f}s")

        # Determinar vencedor baseado em múltiplos critérios
        criterios = []
        if melhor_armijo['iterations'] < melhor_gbb['iterations']:
            criterios.append("Armijo vence em iteracoes")
        if melhor_armijo['final_norm_g'] < melhor_gbb['final_norm_g']:
            criterios.append("Armijo vence em precisao")
        if melhor_armijo['time'] < melhor_gbb['time']:
            criterios.append("Armijo vence em tempo")

        if melhor_gbb['iterations'] < melhor_armijo['iterations']:
            criterios.append("GBB vence em iteracoes")
        if melhor_gbb['final_norm_g'] < melhor_armijo['final_norm_g']:
            criterios.append("GBB vence em precisao")
        if melhor_gbb['time'] < melhor_armijo['time']:
            criterios.append("GBB vence em tempo")

        print(f"\nCriterios de comparacao: {', '.join(criterios)}")

        # Recomendação final baseada no número de vitórias
        if len(criterios) > 0:
            # Contar vitórias de cada algoritmo
            vitorias_armijo = sum(1 for c in criterios if "Armijo" in c)
            vitorias_gbb = sum(1 for c in criterios if "GBB" in c)

            if vitorias_armijo > vitorias_gbb:
                print(f"\nRECOMENDAÇÃO FINAL: Usar Armijo com os parametros otimos sugeridos")
            elif vitorias_gbb > vitorias_armijo:
                print(f"\nRECOMENDAÇÃO FINAL: Usar GBB com os parametros otimos sugeridos")
            else:
                print(f"\nRECOMENDAÇÃO FINAL: Ambos algoritmos sao competitivos - escolher segundo prioridade")

    # ===============================================================
    # ESTATÍSTICAS FINAIS
    # ===============================================================
    print("\n" + "="*70)
    print("ESTATÍSTICAS FINAIS DA BUSCA")
    print("="*70)

    print(f"Problema avaliado: {p.name}")
    print(f"Dimensao: {p.n}")
    print(f"Total de combinacoes avaliadas: {len(all_results)}")
    print(f"Tempo total estimado: {sum(r['time'] for r in all_results):.1f} segundos")

    # Salvar resultados detalhados em arquivo
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    filename = f"parametros_optimizados_{p.name}_{timestamp}.txt"

    with open(filename, 'w') as f:
        f.write(f"PARÂMETROS ÓTIMOS - {p.name}\n")
        f.write("="*50 + "\n")
        f.write(f"Dimensao: {p.n}\n")
        f.write(f"Valor inicial f(x0): {f0:.6e}\n")
        f.write(f"Norma inicial do gradiente: {np.linalg.norm(g0):.3e}\n\n")

        if melhor_armijo:
            f.write(f"ARMIJO ÓTIMO:\n")
            f.write(f"  c1: {melhor_armijo['params']['c1']:.1e}\n")
            f.write(f"  rho: {melhor_armijo['params']['rho']}\n")
            f.write(f"  tau: {melhor_armijo['params']['tau']:.1e}\n")
            f.write(f"  alpha_init: {melhor_armijo['params']['alpha_init']}\n")
            f.write(f"  Resultado: {melhor_armijo['iterations']} iter, ‖∇f‖={melhor_armijo['final_norm_g']:.2e}\n")
            f.write(f"  Tempo: {melhor_armijo['time']:.3f}s\n")
            f.write(f"  Buscas lineares: {melhor_armijo['line_searches']}\n\n")

        if melhor_gbb:
            f.write(f"GBB ÓTIMO:\n")
            f.write(f"  M: {melhor_gbb['params']['M']}\n")
            f.write(f"  tau: {melhor_gbb['params']['tau']:.1e}\n")
            f.write(f"  gamma: {melhor_gbb['params']['gamma']}\n")
            f.write(f"  y_gbb: {melhor_gbb['params']['y_gbb']:.1e}\n")
            f.write(f"  alpha_init: {melhor_gbb['params']['alpha_init']}\n")
            f.write(f"  Resultado: {melhor_gbb['iterations']} iter, ‖∇f‖={melhor_gbb['final_norm_g']:.2e}\n")
            f.write(f"  Tempo: {melhor_gbb['time']:.3f}s\n")
            f.write(f"  Buscas lineares: {melhor_gbb['line_searches']}\n")
            f.write(f"  Buscas espectrais: {melhor_gbb['spectral_searches']}\n")

    print(f"\nResultados salvos em: {filename}")

    return all_results, armijo_results, gbb_results

# ===============================================================
# EXECUÇÃO PRINCIPAL
# ===============================================================
if __name__ == "__main__":
    print("INICIANDO BUSCA DE PARÂMETROS ÓTIMOS")

    try:
        # Lista de problemas disponíveis para teste
        #"ARGLINA", "ARGLINB", "BA-L1SPLS", "BIGGS6", "BROWNAL", "COATING",
        #"FLETCHCR", "GAUSS2LS", "GENROSE", "HAHN1LS", "HEART6LS", "HILBERTB",
        #"HYDCAR6LS", "LANCZOS1LS", "LANCZOS2LS", "LRIJCNN1", "LUKSAN12LS",
        #"LUKSAN16LS", "OSBORNEA", "PALMER1C", "PALMER3C", "PENALTY2", "PENALTY3",
        #"QING", "ROSENBR", "STRTCHDV", "TESTQUAD", "THURBERLS", "TRIGON1",
        #"TOINTGOR
        problema = "ROSENBR"

        # Executar a busca em grade completa
        todos_resultados, resultados_armijo, resultados_gbb = grid_search_optimization(problema)

        if todos_resultados:
            print(f"\nBUSCA CONCLUIDA COM SUCESSO")
            print(f"Verifique o arquivo gerado para os parametros otimos recomendados")
        else:
            print("\nBusca concluida mas sem resultados validos")

    except Exception as e:
        print(f"ERRO CRITICO: {e}")
        import traceback
        traceback.print_exc()